In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
food_150_df = pd.read_csv("../data_preprocessing/merged_food_150_by_Mook.csv")
food_6000_df = pd.read_csv("../data_preprocessing/recipe_6000_by_Mook.csv")
recipe_df = pd.read_csv("../data_preprocessing/recipe_clean.csv")

In [ ]:
food_6000_df.head(5)

In [ ]:
food_6000_df.info()
# food_6000_df.isna().sum()
food_6000_df.dropna(subset="CKG_NM")
food_6000_df.isna().sum()

In [14]:
recipe_df.columns

Index(['CKG_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM',
       'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_MTRL_CN',
       'vegetarian_type'],
      dtype='object')

In [15]:
food_6000_df.columns

Index(['RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT',
       'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM',
       'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN',
       'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT',
       'RCP_IMG_URL', '대표명칭'],
      dtype='object')

In [21]:
# food_150_df에 있지만 recipe_df에 없는 음식 이름 찾기
missing_names = food_150_df[~food_150_df['name'].isin(recipe_df['CKG_NM'])]

# 결과 확인
print(missing_names["name"])

3         곱창구이
14      떡국_만두국
18        시래기국
21         과매기
23          젓갈
27     후라이드_치킨
44          경단
45          꿀떡
48        조개구이
67        꽁치조림
75         닭계장
81     곰탕(설렁탕)
95          약과
96          약식
97          한과
98          멍게
101        산낙지
Name: name, dtype: object


In [ ]:
# !pip install rapidfuzz

In [26]:
from rapidfuzz import fuzz, process

# 누락된 음식 이름 리스트
missing_names = [
    "곱창구이", "떡국_만두국", "시래기국", "과매기", "젓갈", "후라이드_치킨",
    "경단", "꿀떡", "조개구이", "꽁치조림", "닭계장", "곰탕(설렁탕)",
    "약과", "약식", "한과", "멍게", "산낙지"
]

# 레시피 데이터프레임에서 음식 이름 목록 추출
recipe_names = recipe_df['CKG_NM'].tolist()

# 누락된 음식 이름과 레시피 이름 간의 유사도 계산
similar_names = {}
for missing in missing_names:
    matches = process.extract(missing, recipe_names, scorer=fuzz.token_sort_ratio, limit=5)
    similar_names[missing] = matches

# 결과 출력
for missing, matches in similar_names.items():
    print(f"'{missing}'와 유사한 이름들:")
    for match, score, _ in matches:
        print(f"  - {match} (유사도: {score}%)")
    print()

'곱창구이'와 유사한 이름들:
  - 들기름곱창김구이 (유사도: 66.66666666666667%)
  - 귤구이 (유사도: 57.14285714285714%)
  - 김구이 (유사도: 57.14285714285714%)
  - 굴구이 (유사도: 57.14285714285714%)
  - 곱창김 (유사도: 57.14285714285714%)

'떡국_만두국'와 유사한 이름들:
  - 떡만두국 (유사도: 80.0%)
  - 김떡만두국 (유사도: 72.72727272727273%)
  - 사골떡만두국 (유사도: 66.66666666666667%)
  - 만두국 (유사도: 66.66666666666667%)
  - 계란떡만두국 (유사도: 66.66666666666667%)

'시래기국'와 유사한 이름들:
  - 시래기 (유사도: 85.71428571428572%)
  - 들깨시래기국 (유사도: 80.0%)
  - 시래기순대국 (유사도: 80.0%)
  - 시래기된장국 (유사도: 80.0%)
  - 꼬시래기국수 (유사도: 80.0%)

'과매기'와 유사한 이름들:
  - 과메기 (유사도: 66.66666666666667%)
  - 사과깍두기 (유사도: 50.0%)
  - 매콤조기조림 (유사도: 44.44444444444444%)
  - 매콤닭불고기 (유사도: 44.44444444444444%)
  - 용과 (유사도: 40.0%)

'젓갈'와 유사한 이름들:
  - 광어젓갈 (유사도: 66.66666666666667%)
  - 멍게젓갈 (유사도: 66.66666666666667%)
  - 오징어젓갈 (유사도: 57.14285714285714%)
  - 낙지젓갈무침 (유사도: 50.0%)
  - 돼지젓갈조림 (유사도: 50.0%)

'후라이드_치킨'와 유사한 이름들:
  - 후라이드치킨 (유사도: 92.3076923076923%)
  - 닭가슴살순살후라이드치킨 (유사도: 63.1578947368421%)
  - 계란후라이샌드위치 (유사도: 62.5%)
  - 돼지후라이드

In [34]:
# food_150_df의 이름 기준으로 recipe_df의 이름 변경 매핑
# "recipe_df의 기존 음식 이름": "food_150_df의 기준 음식 이름"
name_mapping_ = {
    "떡만두국": "떡국_만두국",
    "시래기된장국": "시래기국",
    "과메기": "과매기",
    "설렁탕": "곰탕(설렁탕)",
    "오징어젓갈": "젓갈",
    "후라이드치킨": "후라이드_치킨",
    "카스테라경단": "경단",
    "꽁치무조림": "꽁치조림",
    "닭개장": "닭계장",
    "약밥": "약식",
    "멍게무침": "멍게",
    "낙지탕탕이": "산낙지",
    # 아래부터는 비슷한 이름이 없어서 눈으로 찾아 바꿈
    # "": "곱창구이",
    # "": "꿀떡",
    # "": "조개구이",
    # "": "약과",
    # "": "한과",
}

In [32]:
# # 'name' 열을 매핑 결과로 업데이트
# recipe_df['CKG_NM'] = recipe_df['CKG_NM'].replace(name_mapping)

# # 결과 확인
# print(recipe_df[recipe_df['CKG_NM'] == '닭계장'])
# print(recipe_df[recipe_df['CKG_NM'] == '꽁치조림'])
# print(food_150_df[food_150_df['name'] == '닭계장'])
# print(food_150_df[food_150_df['name'] == '꽁치조림'])

In [33]:
# # 변경된 이름과 food_150_df의 이름 매칭 여부 확인
# matched_names = recipe_df[recipe_df['CKG_NM'].isin(food_150_df['name'])]
# unmatched_names = recipe_df[~recipe_df['CKG_NM'].isin(food_150_df['name'])]

# print(f"매칭된 이름 수: {len(matched_names)}")
# print(f"매칭되지 않은 이름들: {unmatched_names}")


In [16]:
food_vegitype = recipe_df[['CKG_NM', 'vegetarian_type']]

In [ ]:
food_vegitype

In [35]:
food_6000_df.head(2)

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT,RCP_IMG_URL,대표명칭
0,7037475,실패할일 없는 김장김치양념 /절임배추20kg/ 김장김치 양념비율 황금레시피 /김장담그기,김장김치,kjo2456,김진옥요리가좋다,60777,0,52,절임,일상,채소류,김치/젓갈/장류,안녕하세요 김진옥입니다. 제가 반찬쇼핑몰 10년 운영했던것은 아시죠~ 그때 김장철에...,[재료] 절임배추20kg(1상자 7~9포기)| 무1개(1600g 채썰때 ...,6인분이상,중급,2시간이내,20241031052836,https://recipe1.ezmember.co.kr/cache/recipe/20...,김장김치
1,7022686,국물 닭볶음탕 황금레시피,국물닭볶음탕,81748884,요리하는쮸네,45495,0,35,끓이기,일상,닭고기,메인반찬,국물 닭볶음탕 황금레시피 알려드릴게요 : ),[재료] 절단 닭1kg| 대파1대| 양파1/2개| 당근1/2개...,2인분,아무나,60분이내,20240321110944,https://recipe1.ezmember.co.kr/cache/recipe/20...,국물닭볶음탕


In [26]:
# type(food_6000_df)
# type(food_vegitype)
merged_df = pd.merge(food_6000_df, food_vegitype, how="inner", on="CKG_NM")

In [30]:
only_in_food_6000 = set(food_6000_df['CKG_NM']) - set(food_vegitype['CKG_NM'])
print(f"food_6000_df에만 있는 값:\n{only_in_food_6000}")

food_6000_df에만 있는 값:
{nan, '크래미오이보트'}


In [45]:
# food_vegitype[food_vegitype['CKG_NM'].str.contains('보트', na=False)]
food_6000_df[food_6000_df['CKG_NM'].str.contains('보트', na=False)]

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT,RCP_IMG_URL,대표명칭
1684,7021885,빵없이 통으로 구워서 피자만들기 애호박보트피자,애호박보트피자,gdubu33,만개의레시피,1380,0,70,굽기,일상,채소류,양식,NaN,[재료] 애호박1개| 양파1/2개| 미니파프리카2개| 양송이버섯4...,2인분,아무나,30분이내,20240311142937,https://recipe1.ezmember.co.kr/cache/recipe/20...,애호박보트피자
3657,7033435,편스토랑 진서연 다이어트 레시피 참치마요 에그마요 오이보트 만들기,에그마요오이보트,77721145,써지니,334,0,8,기타,일상,채소류,밥/죽/떡,편스토랑 방송에 나오시는 진서연 다이어트 메뉴 음식으로 아삭아삭한 식감의 오이로 편...,[재료] 오이1개| 쪽파적당히(토핑용)| 현미밥2~3숟가락(또는 쌀...,1인분,아무나,NaN,20240829053438,https://recipe1.ezmember.co.kr/cache/recipe/20...,에그마요오이보트
5440,7038420,채식메뉴 홈브런치 다이어트레시피 핑거푸드 오이요리 크래미 오이보트,크래미오이보트,loveshine707,쿡따라,69,0,4,NaN,NaN,채소류,퓨전,살찔 걱정 없는 크래미 오이보트인데 불 사용없이 간단하게 만들어 먹는 오이요리입니다,[재료] 오이1개| 크래미5개(小)| 단감1/2개| 검정깨약간...,3인분,초급,15분이내,20241112203645,https://recipe1.ezmember.co.kr/cache/recipe/20...,크래미오이보트
